In [3]:
import numpy as np 
import pandas as pd 
import pyodbc
import warnings
from mlxtend.frequent_patterns import apriori, association_rules
warnings.simplefilter('ignore')

In [4]:
servername = 'DESKTOP-LVEKE4S\\SQLEXPRESS01'

In [5]:
dwh_conn = pyodbc.connect('DRIVER={SQL SERVER};SERVER='+servername+';DATABASE=UnitedOutdoors; Trusted_Connection=yes')

In [29]:
orders = pd.read_sql_query('SELECT * FROM order_details', dwh_conn)
products = pd.read_sql_query("select * from product", dwh_conn)
orderproduct = pd.merge(orders, products, on='product_sk', how='inner')
trim = orderproduct[['order_id', 'product_name']]
dummies = pd.get_dummies(trim.loc[:,['product_name']])
split = pd.concat([trim, dummies], axis=1)
split = split.drop(['product_name'], axis=1)

split = split.groupby('order_id').max().reset_index()
split.set_index('order_id', inplace=True)
split

,product_name_AWC Logo Cap,product_name_Alice Mutton,product_name_All-Purpose Bike Stand,product_name_Aniseed Syrup,product_name_Baseball Cap,product_name_Bike Wash - Dissolver,product_name_Boston Crab Meat,product_name_Cable Lock,product_name_Camembert Pierrot,product_name_Carnarvon Tigers,...,product_name_Visor,product_name_Water Bottle - 30 oz.,product_name_Wimmers gute Semmelknödel,"product_name_Women's Mountain Shorts, L","product_name_Women's Mountain Shorts, M","product_name_Women's Mountain Shorts, S","product_name_Women's Tights, L","product_name_Women's Tights, M","product_name_Women's Tights, S",product_name_Zaanse koeken
order_id,,,,,,,,,,,,,,,,,,,,,
2001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2002,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2003,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2004,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2005,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75119,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
75120,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
75121,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [27]:
frq_items = apriori(split, min_support = 0.02, use_colnames = True) 

rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
190,"(product_name_Sport-100 Helmet, Blue, product_...","(product_name_Long-Sleeve Logo Jersey, L)",0.022280,0.049628,0.021733,0.975477,19.655709,0.020627,38.754051,0.970752
188,"(product_name_Sport-100 Helmet, Blue, product_...","(product_name_Sport-100 Helmet, Black)",0.022310,0.091273,0.021733,0.974150,10.672883,0.019697,35.153374,0.926986
131,"(product_name_AWC Logo Cap, product_name_Sport...","(product_name_Long-Sleeve Logo Jersey, L)",0.022978,0.049628,0.022371,0.973580,19.617487,0.021230,35.971574,0.971344
144,"(product_name_AWC Logo Cap, product_name_Sport...","(product_name_Long-Sleeve Logo Jersey, L)",0.022917,0.049628,0.022310,0.973510,19.616076,0.021173,35.876537,0.971280
158,"(product_name_AWC Logo Cap, product_name_Sport...","(product_name_Sport-100 Helmet, Black)",0.022917,0.091273,0.022280,0.972185,10.651363,0.020188,32.670887,0.927368
...,...,...,...,...,...,...,...,...,...,...
214,(product_name_AWC Logo Cap),"(product_name_Sport-100 Helmet, Blue, product_...",0.102656,0.022492,0.021733,0.211709,9.412624,0.019424,1.240034,0.996006
72,(product_name_AWC Logo Cap),"(product_name_Sport-100 Helmet, Black, product...",0.102656,0.021764,0.020003,0.194855,8.953280,0.017769,1.214982,0.989931
40,(product_name_Water Bottle - 30 oz.),"(product_name_Sport-100 Helmet, Black)",0.142298,0.091273,0.021672,0.152304,1.668656,0.008684,1.071996,0.467196
44,(product_name_Water Bottle - 30 oz.),"(product_name_Sport-100 Helmet, Blue)",0.142298,0.093793,0.021399,0.150384,1.603366,0.008053,1.066608,0.438744
